# 1. tf.keras.metrics ：概念
tf.keras.metrics 模块提供了度量指标（Metrics），用于在训练和测试过程中评估模型性能。\
它可以用在：

- `model.compile()` 的 `metrics` 参数中，作为训练和验证的实时评估指标

也可以单独实例化，在训练循环中手动更新和计算

区别于 `Loss`：
- `Loss` 用来优化模型（参与反向传播）
- `Metric` 只做评估（不参与梯度更新）

## 2. 常用指标分类
| 任务类型    | 常用指标                          | 说明                |
| ------- | ----------------------------- | ----------------- |
| **回归**  | `MeanAbsoluteError` (MAE)     | 平均绝对误差            |
|         | `MeanSquaredError` (MSE)      | 平均平方误差            |
|         | `RootMeanSquaredError` (RMSE) | 均方根误差             |
| **二分类** | `BinaryAccuracy`              | 二分类准确率            |
|         | `Precision`                   | 精确率               |
|         | `Recall`                      | 召回率               |
|         | `AUC`                         | ROC 曲线下的面积        |
| **多分类** | `CategoricalAccuracy`         | 多分类准确率（独热编码标签）    |
|         | `SparseCategoricalAccuracy`   | 多分类准确率（整数标签）      |
| **通用**  | `Accuracy`                    | 自动选择二分类或多分类方式     |
|         | `TopKCategoricalAccuracy`     | 前 K 个预测中包含真实标签的比例 |
|         | `MeanIoU`                     | 交并比（图像分割常用）       |

## 3. tf.keras.metrics.Metric 基本参数表
大多数 Keras 指标类（如 tf.keras.metrics.Accuracy、tf.keras.metrics.Precision）继承自 tf.keras.metrics.Metric，它们通常有以下构造参数：
| 参数               | 类型                            | 说明                         |
| ---------------- | ----------------------------- | -------------------------- |
| **`name`**       | `str`                         | 指标名称，默认自动生成                |
| **`dtype`**      | `str` (`float32` / `float64`) | 计算时的数据类型                   |
| **`thresholds`** | `list` 或 `float`              | 用于二分类的概率阈值（如 `0.5`）        |
| **`top_k`**      | `int`                         | 仅在前 K 个预测中寻找正确类别（Top-K 精度） |

## 4. 两种使用方式
### 方式 1：在 model.compile() 中指定

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=[
        tf.keras.metrics.CategoricalAccuracy(name='acc'),
        tf.keras.metrics.AUC(name='auc')
    ]
)


### 方式 2：手动创建并更新

In [ ]:
import tensorflow as tf

m = tf.keras.metrics.Accuracy()

# 模拟真实值与预测值
y_true = [1, 2, 3, 4]
y_pred = [0, 2, 3, 4]

m.update_state(y_true, y_pred)  # 更新状态
print("准确率：", m.result().numpy())  # 输出: 0.75

m.reset_state()  # 清空状态（下一轮计算前必须）


## 5. 运行机制
每个指标类内部维护一个或多个状态变量（`state variables`），例如 count、total 等:
- 1、`update_state`(y_true, y_pred)：接收真实值和预测值，更新内部状态（累计正确数、总数等）
- 2、`result()`：计算并返回当前的指标值
- 3、`reset_state()`：清空状态，为下一轮计算做准备

---
## 6. 注意事项
- 在 回归任务 中，指标一般不会影响优化过程，仅作为监控手段

- 训练日志中的 `loss` 和 `metrics` 是分别计算的，`loss` 参与梯度计算，`metrics` 不参与

- 自定义指标可以继承 `tf.keras.metrics.Metric` 并实现三个方法：`__init__()`、`update_state()`、`result()`